In [1]:
import os

In [2]:
%pwd

'd:\\archive\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\archive\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [12]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [15]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [18]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-02-23 13:42:13,326: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-23 13:42:13,335: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-23 13:42:13,338: INFO: common: created directory at: artifacts]
[2024-02-23 13:42:13,340: INFO: common: created directory at: artifacts\training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/20
17/17 [==============================] - 52s 3s/step - loss: 13.2832 - accuracy: 0.5367 - val_loss: 16.4271 - val_accuracy: 0.3906
Epoch 2/20
17/17 [==============================] - 51s 3s/step - loss: 14.5044 - accuracy: 0.4981 - val_loss: 16.3491 - val_accuracy: 0.6094
Epoch 3/20
17/17 [==============================] - 50s 3s/step - loss: 10.6582 - accuracy: 0.6371 - val_loss: 0.0922 - val_accuracy: 0.9844
Epoch 4/20
17/17 [==============================] - 50s 3s/step - loss: 5.6102 - accuracy: 0.6911 - val_loss: 14.0915 - val_accuracy: 0.6094
Epoch 5/20
17/17 [=====

UnknownError: Graph execution error:

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\data_ingestion\\Chest-CT-Scan-data\\normal\\5.png'
Traceback (most recent call last):

  File "c:\Users\Shreya\anaconda3\envs\chest\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 267, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "c:\Users\Shreya\anaconda3\envs\chest\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\Shreya\anaconda3\envs\chest\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\Shreya\anaconda3\envs\chest\Lib\site-packages\keras\engine\data_adapter.py", line 902, in wrapped_generator
    for data in generator_fn():

  File "c:\Users\Shre